In [28]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch, torchvision
import torchvision.transforms as T
from torch import nn
from torch import optim
# from torchvision.transforms import ToTensor
import torch.nn.functional as F

import sys
sys.path.append("./libs/ACDA")
import Conv_DCFD as acda

In [29]:
# Create image used in  --> DELETE THIS CELL LATER!
# from teh bottom of 
layer = acda.Conv_DCFD(3, 10, kernel_size=3, inter_kernel_size=5, padding=1, stride=2, bias=True).cuda()
data = torch.randn(10 , 3, 224, 224).cuda()
print(layer(data).shape)

dropout2d parameters:  torch.Size([10, 3, 224, 224]) 0.0 True
Shapes of the unfold parameters:  torch.Size([10, 3, 224, 224]) 3 2
view parameters:  10 3 9 112 112
torch.Size([10, 10, 112, 112])


### LeNet
The LeNet network only has 2 layers

References for LetNet implementations:
https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/


In [30]:
# def create_lenet():
#     model = nn.Sequential(
#         nn.Conv2d(1, 6, 5, padding=2),
#         nn.ReLU(),
#         nn.AvgPool2d(2, stride=2),
#         nn.Conv2d(6, 16, 5, padding=0),
#         nn.ReLU(),
#         nn.AvgPool2d(2, stride=2),
#         nn.Flatten(),
#         nn.Linear(400, 120),
#         nn.ReLU(),
#         nn.Linear(120, 84),
#         nn.ReLU(),
#         nn.Linear(84, 10)
#     )
#     return model
# https://medium.com/@krishna.ramesh.tx/training-a-cnn-to-distinguish-between-mnist-digits-using-pytorch-620f06aa9ffa

#Defining the convolutional neural network

class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

class LeNet5_ACDA(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5_ACDA, self).__init__()
        self.layer1 = nn.Sequential(
            # nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            acda.Conv_DCFD(3, 6, kernel_size=5, stride=1, padding=2),
            # nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            # nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            acda.Conv_DCFD(6, 16, kernel_size=5, stride=1, padding=2),
            # nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        # YVETTE - need to get rid of the padding that was added!!!
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

leNet = LeNet5(num_classes=10)
leNet_acda = LeNet5_ACDA(num_classes=10)

In [31]:

criterion = nn.CrossEntropyLoss()
leNet_optimizer = optim.SGD(leNet.parameters(), lr=0.001, momentum=0.9)
leNet_acda_optimizer = optim.SGD(leNet.parameters(), lr=0.001, momentum=0.9)

References for using CIFAR-10 data:
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [32]:
# Using CIFAR-10 data
batch_size = 6

transform = T.Compose(
    [T.ToTensor(),
     T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Train network
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        leNet_optimizer.zero_grad()

        # forward + backward + optimize

        print(' shape of inputs: ', inputs.shape)
        outputs = leNet_acda(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        leNet_optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


Files already downloaded and verified
Files already downloaded and verified
 shape of inputs:  torch.Size([6, 3, 32, 32])
dropout2d parameters:  torch.Size([6, 3, 32, 32]) 0.0 True
Shapes of the unfold parameters:  torch.Size([6, 3, 32, 32]) 5 1
view parameters:  6 3 25 32 32
dropout2d parameters:  torch.Size([6, 6, 16, 16]) 0.0 True
Shapes of the unfold parameters:  torch.Size([6, 6, 16, 16]) 5 1
view parameters:  6 6 25 16 16


RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x1024 and 400x120)

In [ ]:
# Test on whole dataset
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = leNet(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')